<a href="https://colab.research.google.com/github/jminango20/DataCleaning/blob/master/Inconsistent_Data_Entry_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Efficiently fix typos in your data

In [2]:
#Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
#Libraries
import pandas as pd
import numpy as np

In [33]:
professores = pd.read_csv('/content/drive/My Drive/Colab Notebooks/DataCleaning/datasets_819513_1402182_pakistan_intellectual_capital.csv')
professores.head()

,Unnamed: 0,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
0,2,3,Dr. Abdul Basit,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,Software Engineering & DBMS,NaN
1,4,5,Dr. Waheed Noor,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,DBMS,NaN
2,5,6,Dr. Junaid Baber,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"Information processing, Multimedia mining",NaN
3,6,7,Dr. Maheen Bakhtyar,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"NLP, Information Retrieval, Question Answering...",NaN
4,24,25,Samina Azim,Sardar Bahadur Khan Women's University,Computer Science,Balochistan,Lecturer,BS,Balochistan University of Information Technolo...,Pakistan,2005.0,VLSI Electronics DLD Database,NaN


Say we're interested in cleaning up the "Country" column to make sure there's no data entry inconsistencies in it. We could go through and check each row by hand, of course, and hand-correct inconsistencies when we find them. There's a more efficient way to do this, though!

In [34]:
# get all the unique values in the 'Country' column
countries = professores['Country'].unique()

# sort them alphabetically and then take a closer look
countries.sort()
print(countries)

[' Germany' ' New Zealand' ' Sweden' ' USA' 'Australia' 'Austria' 'Canada'
 'China' 'Finland' 'France' 'Greece' 'HongKong' 'Ireland' 'Italy' 'Japan'
 'Macau' 'Malaysia' 'Mauritius' 'Netherland' 'New Zealand' 'Norway'
 'Pakistan' 'Portugal' 'Russian Federation' 'Saudi Arabia' 'Scotland'
 'Singapore' 'South Korea' 'SouthKorea' 'Spain' 'Sweden' 'Thailand'
 'Turkey' 'UK' 'USA' 'USofA' 'Urbana' 'germany']


Just looking at this, I can see some problems due to inconsistent data entry: ' Germany', and 'germany', for example, or ' New Zealand' and 'New Zealand'.

The first thing I'm going to do is make everything lower case (I can change it back at the end if I like) and remove any white spaces at the beginning and end of cells. Inconsistencies in capitalizations and trailing white spaces are very common in text data and you can fix a good 80% of your text data entry inconsistencies by doing this.

In [35]:
# convert to lower case
professores['Country'] = professores['Country'].str.lower()
professores['Country'].unique()

array(['thailand', 'pakistan', 'germany', 'austria', 'australia', 'uk',
       'china', 'france', 'usofa', 'southkorea', 'malaysia', 'sweden',
       'italy', 'canada', 'norway', 'ireland', 'new zealand', 'urbana',
       'portugal', 'russian federation', 'usa', 'finland', ' usa',
       'netherland', ' germany', ' sweden', ' new zealand', 'greece',
       'turkey', 'south korea', 'macau', 'singapore', 'spain', 'japan',
       'hongkong', 'saudi arabia', 'mauritius', 'scotland'], dtype=object)

In [36]:
# remove trailing white spaces
professores['Country'] = professores['Country'].str.strip()
professores['Country'].unique()

array(['thailand', 'pakistan', 'germany', 'austria', 'australia', 'uk',
       'china', 'france', 'usofa', 'southkorea', 'malaysia', 'sweden',
       'italy', 'canada', 'norway', 'ireland', 'new zealand', 'urbana',
       'portugal', 'russian federation', 'usa', 'finland', 'netherland',
       'greece', 'turkey', 'south korea', 'macau', 'singapore', 'spain',
       'japan', 'hongkong', 'saudi arabia', 'mauritius', 'scotland'],
      dtype=object)

## Use fuzzy matching to correct inconsistent data entry

Alright, let's take another look at the 'Country' column and see if there's any more data cleaning we need to do.

In [37]:
# get all the unique values in the 'Country' column
countries = professores['Country'].unique()

# sort them alphabetically and then take a closer look
countries.sort()
print(countries)

['australia' 'austria' 'canada' 'china' 'finland' 'france' 'germany'
 'greece' 'hongkong' 'ireland' 'italy' 'japan' 'macau' 'malaysia'
 'mauritius' 'netherland' 'new zealand' 'norway' 'pakistan' 'portugal'
 'russian federation' 'saudi arabia' 'scotland' 'singapore' 'south korea'
 'southkorea' 'spain' 'sweden' 'thailand' 'turkey' 'uk' 'urbana' 'usa'
 'usofa']


It does look like there is another inconsistency: 'southkorea' and 'south korea' should be the same.

We're going to use the fuzzywuzzy package to help identify which strings are closest to each other.

**Fuzzy matching:** The process of automatically finding text strings that are very similar to the target string. In general, a string is considered "closer" to another one the fewer characters you'd need to change if you were transforming one string into another. So "apple" and "snapple" are two changes away from each other (add "s" and "n") while "in" and "on" and one change away (rplace "i" with "o"). You won't always be able to rely on fuzzy matching 100%, but it will usually end up saving you at least a little time.

**Fuzzywuzzy** returns a ratio given two strings. The closer the ratio is to 100, the smaller the edit distance between the two strings. Here, we're going to get the ten strings from our list of cities that have the closest distance to "d.i khan".

In [38]:
!pip install fuzzywuzzy

In [39]:
# helpful modules
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

In [40]:
# get the top 10 closest matches to "south korea"
matches = fuzzywuzzy.process.extract(query="south korea",choices=countries,scorer=fuzzywuzzy.fuzz.token_sort_ratio, limit=10)

In [41]:
# take a look at them
print(matches)

[('south korea', 100), ('southkorea', 48), ('saudi arabia', 43), ('norway', 35), ('ireland', 33), ('portugal', 32), ('singapore', 30), ('netherland', 29), ('macau', 25), ('usofa', 25)]


We can see that two of the items in the cities are very close to "south korea": "south korea" and "southkorea". Let's replace all rows in our "Country" column that have a ratio of > 47 with "south korea".

**To do this, I'm going to write a function.**

In [42]:
# function to replace rows in the provided column of the provided dataframe
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 47):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")

In [43]:
# use the function we just wrote to replace close matches to "south korea" with "south korea"
replace_matches_in_column(df=professores,column='Country',string_to_match='south korea')

All done!


In [44]:
professores['Country'].unique()

array(['thailand', 'pakistan', 'germany', 'austria', 'australia', 'uk',
       'china', 'france', 'usofa', 'south korea', 'malaysia', 'sweden',
       'italy', 'canada', 'norway', 'ireland', 'new zealand', 'urbana',
       'portugal', 'russian federation', 'usa', 'finland', 'netherland',
       'greece', 'turkey', 'macau', 'singapore', 'spain', 'japan',
       'hongkong', 'saudi arabia', 'mauritius', 'scotland'], dtype=object)

# 1) Examine another column

Write code below to take a look at all the unique values in the "Graduated from" column.

In [45]:
graduated_form = professores['Graduated from'].unique()
print(graduated_form)

['Asian Institute of Technology'
 'Balochistan University of Information Technology, Engineering and Management Sciences'
 'University of Balochistan' "Sardar Bahadur Khan Women's University"
 'SRH Hochschule Heidelberg'
 'Institute of Business Administration,Karachi' 'DUET,Karachi'
 'University of Turbat' 'University of Vienna' 'Monash University'
 'University of Stirling' 'Chinese Academy of Sciences'
 'University of Innsbruck' 'Vienna University of Technology'
 'University of Paris-Est' 'The University of Cambridge'
 'Harbin Institute of Technology' 'University of Nice, Sophia Antipolis'
 'The University of York' 'Galilée - Université Paris 13'
 'University of Bedfordshire' 'North Dakota State University'
 'Kyungpook National University' 'The University of Manchester'
 'National University of Sciences and Technology'
 'FAST– National University of Computer and Emerging Sciences'
 'Capital University of Science & Technology' 'Gomal University'
 'University of Malaya' 'KTH Royal Insti

# 2) Do some text pre-processing

Convert every entry in the "Graduated from" column in the professors DataFrame to remove white spaces at the beginning and end of cells.

In [46]:
professores['Graduated from'] = professores['Graduated from'].str.strip()

In [26]:
professores['Graduated from'].unique()

array(['Asian Institute of Technology',
       'Balochistan University of Information Technology, Engineering and Management Sciences',
       'University of Balochistan',
       "Sardar Bahadur Khan Women's University",
       'SRH Hochschule Heidelberg',
       'Institute of Business Administration,Karachi', 'DUET,Karachi',
       'University of Turbat', 'University of Vienna',
       'Monash University', 'University of Stirling',
       'Chinese Academy of Sciences', 'University of Innsbruck',
       'Vienna University of Technology', 'University of Paris-Est',
       'The University of Cambridge', 'Harbin Institute of Technology',
       'University of Nice, Sophia Antipolis', 'The University of York',
       'Galilée - Université Paris 13', 'University of Bedfordshire',
       'North Dakota State University', 'Kyungpook National University',
       'The University of Manchester',
       'National University of Sciences and Technology',
       'FAST– National University of Computer

## 3) Continue working with countries

In the tutorial, we focused on cleaning up inconsistencies in the "Country" column. Run the code cell below to view the list of unique values that we ended with.

In [47]:
# get all the unique values in the 'City' column
countries = professores['Country'].unique()

# sort them alphabetically and then take a closer look
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea', 'spain',
       'sweden', 'thailand', 'turkey', 'uk', 'urbana', 'usa', 'usofa'],
      dtype=object)

Take another look at the "Country" column and see if there's any more data cleaning we need to do.

It looks like 'usa' and 'usofa' should be the same country. Correct the "Country" column in the dataframe so that 'usofa' appears instead as 'usa'.

Use the most recent version of the DataFrame (with the whitespaces at the beginning and end of cells removed) from question 2.

In [48]:
matches = fuzzywuzzy.process.extract("usa", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
print(matches)

[('usa', 100), ('usofa', 75), ('austria', 60), ('australia', 50), ('spain', 50), ('urbana', 44), ('uk', 40), ('malaysia', 36), ('pakistan', 36), ('portugal', 36)]


In [49]:
replace_matches_in_column(df=professores,column='Country',string_to_match='usa',min_ratio=70)

All done!


In [50]:
professores['Country'].unique()

array(['thailand', 'pakistan', 'germany', 'austria', 'australia', 'uk',
       'china', 'france', 'usa', 'south korea', 'malaysia', 'sweden',
       'italy', 'canada', 'norway', 'ireland', 'new zealand', 'urbana',
       'portugal', 'russian federation', 'finland', 'netherland',
       'greece', 'turkey', 'macau', 'singapore', 'spain', 'japan',
       'hongkong', 'saudi arabia', 'mauritius', 'scotland'], dtype=object)